<a href="https://colab.research.google.com/github/sidd6p/AWS-ML-Product_Length/blob/main/src/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load Data

In [4]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials


# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [5]:
link="https://drive.google.com/file/d/16ZHfZwv8VV2jGWRrJgfqalAOcDc7gTJH/view?usp=share_link"
import pandas as pd

# to get the id part of the file
id = link.split("/")[-2]

downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('train_this.csv')


In [6]:
raw_data = pd.read_csv('train_this.csv')

In [7]:
raw_data

,PRODUCT_ID,TITLE,BULLET_POINTS,DESCRIPTION,PRODUCT_TYPE_ID,PRODUCT_LENGTH,CATEGORY
0,22,"Quantum Mechanics, 2nd Edition (Special Indian...",NaN,NaN,6322,787.400000,5
1,27,"The Lion, the Witch and the Wardrobe (The Chro...",NaN,NaN,42,417.322000,4
2,30,My Cousin Rachel,NaN,NaN,99,417.322000,4
3,33,Coriolanus,NaN,NaN,113,417.322000,4
4,37,The Crucible,NaN,NaN,113,417.322000,4
...,...,...,...,...,...,...,...
118147,2999928,Beat Top Model Car 9 inch Full Touch Screen An...,[2GB RAM/16GB ROM With 1 Year Warranty: Develo...,Product Version: Hi-Fi Version with Vertical o...,2108,984.251967,5
118148,2999930,Black Panther: 2018 (DVD),DVD MOVIE,"After his father's death, T'Challa returns hom...",3,1102.362204,6
118149,2999937,Customized Happy Anniversary Wooden Photo Fram...,"[Photo Quantity: 4 Images Required,Dimension: ...","Express Your Love & Feelings On Valentines, An...",12228,1259.842518,6
118150,2999946,LAKWAL Handcrafted Brass Golden Kamdhenu Cow w...,"[Measures Length: 17 CM, Width:12 CM, Height:1...",Handcrafted Large Gold Elegant Kamdhenu Cow an...,5506,669.291338,5


In [24]:
texts = ['TITLE', 'BULLET_POINTS', 'DESCRIPTION']

# Clean Data

In [16]:
import numpy as np
import pandas as pd
import nltk
import re

from sklearn import preprocessing
from nltk.corpus import stopwords

In [14]:
nltk.download('stopwords')
stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [19]:
raw_data = raw_data.dropna(how='any')
raw_data = raw_data.sample(frac=1, random_state=42)

In [20]:
def remove_punctuation(text):
    final = "".join(u for u in text if u not in ("?", ".", ";", ":", "!", '"', "'"))
    return final

In [21]:
def preprocess_text(text):
    text = re.sub(r'[^\w\s]', '', text)
    text = text.lower()
    text = " ".join([word for word in text.split() if word not in stop_words])
    return text

In [23]:
for text in texts:
  raw_data[text] = raw_data[text].astype(str)
  raw_data[text] = raw_data[text].apply(remove_punctuation)
  raw_data[text] = raw_data[text].apply(preprocess_text)
  raw_data[text] = raw_data[text].apply(lambda x: ' '.join(x.split()))  # Remove extra whitespaces
  raw_data[text] = raw_data[text].str.replace('\d+', '')  # Remove digits
  raw_data[text] = raw_data[text].str.replace('[^\x00-\x7F]+', '')  # Remove non-ASCII characters

<ipython-input-23-b1727534fe32>:6: FutureWarning: The default value of regex will change from True to False in a future version.
  raw_data[text] = raw_data[text].str.replace('\d+', '')  # Remove digits
<ipython-input-23-b1727534fe32>:7: FutureWarning: The default value of regex will change from True to False in a future version.
  raw_data[text] = raw_data[text].str.replace('[^\x00-\x7F]+', '')  # Remove non-ASCII characters


In [25]:
raw_data

,PRODUCT_ID,TITLE,BULLET_POINTS,DESCRIPTION,PRODUCT_TYPE_ID,PRODUCT_LENGTH,CATEGORY
48838,1238161,brass glass nautical finder compass briton lon...,brass made products made solid brass carefully...,brass glass nautical finder compass briton lon...,5506,196.850393,3
75892,1924942,micra powermat acupressure magnetic foot relie...,effective health cure right super india store ...,effective health cure right micra brings valua...,8453,1181.102361,6
60222,1526668,xpand quick release roundlacing tie shoelaces ...,simple comfortable never tie shoes turn shoe s...,xpand quick release roundlacing system one kin...,3101,5200.000000,8
116558,2960925,renue lifestyle brass waste coupling wash basi...,corrosion resistant material wash basin waster...,pthe design detailing bathroom something bring...,10279,433.070866,4
74488,1889972,real rugs kids pets friendly bella woollen rug...,exciting handmade rugs make statement contempo...,pexperience stunning beauty charm rug collecti...,1636,7200.000000,8
...,...,...,...,...,...,...,...
107187,2723116,stop fashion womens crape multicolor digital ...,material american crepe th sleevesprint typed...,crepe stitched size smlxlxxlxl fancy kurtis s...,2916,1000.000000,5
118075,2998179,alstonia multi color premium digital print cus...,low covers invisible zipper hidden zippers mak...,defined printed surface design cushion cover s...,7256,1574.803148,6
98916,2511436,bbq gloves heat resistant grilling gloves blac...,premium triple material heatresistant gloves c...,bbq grill gloves,5651,1496.000000,6
33149,845187,cate bulk cable awg blue solid,utp cate,cate bulk cable awgblue solid,4,827.000000,5


# MODEL 0 - NAIVE BAYES

In [26]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

In [27]:
label_encoder = preprocessing.LabelEncoder()

# Encode labels in column 'CATEGORY'.
raw_data['CATEGORY']= label_encoder.fit_transform(raw_data['CATEGORY'])

In [28]:
splitter = int(len(raw_data)*.8)
train, test = raw_data[:splitter], raw_data[splitter:]

In [32]:
MODELS = dict()
SCRS = dict()

In [33]:
for text in texts:
  X_train, y_train = train[text], train['CATEGORY']
  X_val, y_val = test[text], test['CATEGORY']
  model = Pipeline([
                    ('tfidf', TfidfVectorizer()),
                    ('clf', MultinomialNB(alpha = 1))
  ])
  model.fit(X_train, y_train)
  MODELS[text] = model
  model_score = MODELS[text].score(X_val, y_val)
  model_score * 100
  SCRS[text] = MODELS[text].predict(X_val)

In [34]:
MODELS

{'TITLE': Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB(alpha=1))]),
 'BULLET_POINTS': Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB(alpha=1))]),
 'DESCRIPTION': Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB(alpha=1))])}

In [35]:
SCRS

{'TITLE': array([5, 5, 5, ..., 5, 5, 6]),
 'BULLET_POINTS': array([5, 5, 5, ..., 5, 5, 5]),
 'DESCRIPTION': array([5, 5, 5, ..., 5, 5, 5])}

# Model 1 - NN

In [ ]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

In [36]:
a = np.array(SCRS['TITLE'])
b = np.array(SCRS['BULLET_POINTS'])
c = np.array(SCRS['DESCRIPTION'])
d = np.array(test['PRODUCT_TYPE_ID'])
e = np.array(test['PRODUCT_LENGTH'], dtype=int)

In [37]:
a, b, c, d, e

(array([5, 5, 5, ..., 5, 5, 6]),
 array([5, 5, 5, ..., 5, 5, 5]),
 array([5, 5, 5, ..., 5, 5, 5]),
 array([2851, 2124, 6978, ..., 5651,    4, 2640]),
 array([1102,  669, 2400, ..., 1496,  827, 1500]))

In [38]:
# Define the model architecture
model = Sequential()
model.add(Dense(256, activation='relu', input_dim=4))
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='linear'))

# Optimize
optimizer = Adam(lr=0.001)

# Compile the model
model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mae', 'accuracy'])

# Train the model
model.fit(x=np.array([a, b, c, d]).T, y=e, epochs=100, validation_split=0.2)

# Make predictions
predictions = model.predict(np.array([a, b, c, d]).T)

Epoch 1/100
273/273 [==============================] - 9s 5ms/step - loss: 16173135872.0000 - mae: 3784.5747 - accuracy: 0.0014 - val_loss: 125388224.0000 - val_mae: 1771.7257 - val_accuracy: 4.5935e-04
Epoch 2/100
273/273 [==============================] - 1s 4ms/step - loss: 16172493824.0000 - mae: 4019.0503 - accuracy: 0.0014 - val_loss: 142116880.0000 - val_mae: 4096.7563 - val_accuracy: 4.5935e-04
Epoch 3/100
273/273 [==============================] - 2s 7ms/step - loss: 16175026176.0000 - mae: 4108.7432 - accuracy: 0.0014 - val_loss: 131453760.0000 - val_mae: 2954.0718 - val_accuracy: 4.5935e-04
Epoch 4/100
273/273 [==============================] - 2s 7ms/step - loss: 16173806592.0000 - mae: 4114.7153 - accuracy: 0.0013 - val_loss: 125068624.0000 - val_mae: 1478.4033 - val_accuracy: 4.5935e-04
Epoch 5/100
273/273 [==============================] - 2s 7ms/step - loss: 16173773824.0000 - mae: 4101.0249 - accuracy: 0.0014 - val_loss: 125056224.0000 - val_mae: 1617.3497 - val_accura

In [39]:
predictions

array([[1396.5579 ],
       [1160.3962 ],
       [2674.7927 ],
       ...,
       [2265.8494 ],
       [ 752.58026],
       [1356.2566 ]], dtype=float32)

In [63]:
# Scoring
MAPE = 100 * np.mean(np.abs(e - predictions) / predictions)
score = max( 0 , 100*(1-MAPE))
print(MAPE, score)

210.1806594112146 0
